In [2]:
import pandas as pd
import numpy as np
import io
import sys
import os
import urllib.request
from tqdm import tqdm
from os import listdir
from PIL import Image
import glob

In [3]:
CSV_FILE = os.path.abspath('../data/MovieGenre.csv')

df = pd.read_csv(CSV_FILE, encoding='ISO-8859-1')
df.head()

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...


In [4]:
df.shape

(40107, 6)

In [5]:
# How many images actually downloaded
IMAGES_PATH = os.path.abspath('../data/IMDB_IMAGES/')

image_list = []
for file in glob.glob(IMAGES_PATH + "*.jpg"):
    image_list.append(file)
    
print("Number of files found:", len(image_list))

Number of files found: 0


In [6]:
# Resolve mis-match
# Create new dataframe with paths of found images and fill with corresponding metadata from dataframe. 

image_paths = []
imdb_id = []
genres = []
titles = []

for file in glob.glob(IMAGES_PATH + "*.jpg"):
    
    #img_id = file.split('.')[0]    
    img_id = file[file.find('/')+1 : file.find('.')]
    title = df[df["imdbId"] == int(img_id)]["Title"].values[0]
    genre = df[df["imdbId"] == int(img_id)]["Genre"].values[0]
    
    image_paths.append(file)
    imdb_id.append(img_id)
    genres.append(genre)
    titles.append(title)
    
    
df_2 = pd.DataFrame({'imdbId': imdb_id, 'Genre': genres, 'Title': titles, 'Image_Paths': image_paths})


In [7]:
print(df_2.shape)
df_2.head()

(0, 4)


,imdbId,Genre,Title,Image_Paths


In [8]:
columnsTitles = ['Image_Paths','imdbId','Genre','Title']
df_2 = df_2.reindex(columns=columnsTitles)
df_2.head()

,Image_Paths,imdbId,Genre,Title


In [9]:
PATH_TO_DATA = os.path.abspath('../data/')
df_2.to_csv(os.path.join(PATH_TO_DATA, "MovieGenre_final.csv"), index = None)

In [10]:
# Investigating which of the Imdb_Id of the original csv, did not have a corresponding image

Original_ID_list = np.asarray(df['imdbId']).astype(int)
ID_with_images_found = np.asarray(df_2['imdbId']).astype(int)

#Original lengths of the two arrays 
print("ORIGINAL:", len(Original_ID_list))
print("TRIMMED:", len(ID_with_images_found))

#First compare the unique values of these two arrays
uniq1, counts1 = np.unique(Original_ID_list, return_counts=True)
print("#unique ID values in ORIGINAL:", len(uniq1))

uniq2, counts2 = np.unique(ID_with_images_found, return_counts=True)
print("#unique ID values in TRIMMED:", len(uniq2))

ORIGINAL: 40107
TRIMMED: 0
#unique ID values in ORIGINAL: 39514
#unique ID values in TRIMMED: 0


In [11]:
#Find the ID values which are non-unique
non_unique_id = []
for i in range(len(counts1)):
    
    if (counts1[i]>1):
        #print(uniq1[i], "with freq=", counts1[i])
        non_unique_id.append(uniq1[i])
        
#print(non_unique_id)

#Find rows where the column value of imdb is among these non_unique_id values
for i in range(len(non_unique_id)):
    
    print(df.loc[df['imdbId'] == int(non_unique_id[i])])

       imdbId                          Imdb Link             Title  \
12409   11358  http://www.imdb.com/title/tt11358  Just Pals (1920)   
13003   11358  http://www.imdb.com/title/tt11358  Just Pals (1920)   

       IMDB Score                 Genre  \
12409         6.6  Comedy|Drama|Western   
13003         6.6  Comedy|Drama|Western   

                                                  Poster  
12409  https://images-na.ssl-images-amazon.com/images...  
13003  https://images-na.ssl-images-amazon.com/images...  
       imdbId                          Imdb Link                     Title  \
12748   11439  http://www.imdb.com/title/tt11439  The Mark of Zorro (1920)   
13342   11439  http://www.imdb.com/title/tt11439  The Mark of Zorro (1920)   

       IMDB Score                      Genre  \
12748         7.3  Adventure|Romance|Western   
13342         7.3  Adventure|Romance|Western   

                                                  Poster  
12748  https://images-na.ssl-images-amazon.

In [ ]:
#Therefore, we conclude that the duplicate IMDB IDs are now removed and we have only unique IDs with their 
#corresponding posters 
#Next : Organise_data with "MovieGenre_final.csv"